In [6]:
import torch 
import numpy as np
import math
import pandas as pd 
import torch.nn as nn
import torch.utils.data as data
import torch.nn.functional as F

In [7]:
import torch 
import numpy as np
import math
import pandas as pd 
import torch.nn as nn
import torch.utils.data as data
import torch.nn.functional as F

In [11]:
DATASET_DIR = "C:/Users/db/Desktop/기업-수강ncf/NCF 졸업생 데이터"
train= pd.read_csv(DATASET_DIR+ "/NCF 졸업생 데이터 (1부).csv", sep='|')[['sex','ccd','bzc_cd','grup_cd']]
test = pd.read_csv(DATASET_DIR+ "/NCF 졸업생 데이터 (2부).csv", sep='|')[['sex','ccd','bzc_cd','grup_cd']]

C:\Users\db\AppData\Local\Temp\ipykernel_32072\1374857115.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  train= pd.read_csv(DATASET_DIR+ "/NCF 졸업생 데이터 (1부).csv", sep='|')[['sex','ccd','bzc_cd','grup_cd']]
C:\Users\db\AppData\Local\Temp\ipykernel_32072\1374857115.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv(DATASET_DIR+ "/NCF 졸업생 데이터 (2부).csv", sep='|')[['sex','ccd','bzc_cd','grup_cd']]


In [9]:
train

,stid,sex,ccd,rk,yy,hk,grup_cd,s_avg,hcd,hired_yy,co,bzc_cd
0,201313917,1,240301252,7,2015,1,4480022,2.5,1,2017,이천시청 정구선수단,84113
1,201313917,1,240301252,7,2015,1,4480058,4.5,1,2017,이천시청 정구선수단,84113
2,201313917,1,240301252,7,2015,1,4480993,0.0,1,2017,이천시청 정구선수단,84113
3,201313917,1,240301252,7,2015,1,4482010,4.5,1,2017,이천시청 정구선수단,84113
4,201314021,1,1150954000,7,2015,1,1200126,3.5,1,2017,(주)에스디,42311
...,...,...,...,...,...,...,...,...,...,...,...,...
709976,201314030,2,1150954000,7,2015,1,4393014,4.0,1,2018,종근당건강(주),10797
709977,201314030,2,1150954000,7,2015,1,4393035,2.5,1,2018,종근당건강(주),10797
709978,201314030,2,1150954000,7,2015,1,4393036,3.0,1,2018,종근당건강(주),10797
709979,201314030,2,1150954000,7,2015,1,4393039,2.5,1,2018,종근당건강(주),10797


In [12]:
rating_list = np.concatenate([train.values, test.values])
n_rating_list = []
for a in rating_list:
    sex,ccd,bzc_cd,grup_cd = a
    n_rating_list.append([sex,ccd,bzc_cd,grup_cd])
rating_list = np.array(n_rating_list)

In [13]:
rating_list

array([['1', '240301252', '84113', '4480022'],
       ['1', '240301252', '84113', '4480058'],
       ['1', '240301252', '84113', '4480993'],
       ...,
       ['2', '1150954000', '21102', '1300012'],
       ['2', '1150954000', '21102', '4393039'],
       ['2', '1150954000', '21102', '4393993']], dtype='<U11')

In [14]:
from sklearn.model_selection import train_test_split
# 모든 유저와 모든 item을 담음
unq_ccd, unq_bzccd,unq_grup_cd = [np.unique(rating_list[:, n+1]) for n in range(3)]
# user와 item의 개수 넣음
num_ccd, num_bzc, num_grup = len(unq_ccd), len(unq_bzccd),len(unq_grup_cd)
print('num_ccd:{} num_bzc:{} num_item:{}'.format(num_ccd, num_bzc, num_grup))
train_list = np.array(train)
test_list = np.array(test)
ccd2idx, bzcidx ,grupidx = {}, {},{}
for i, j in enumerate(unq_ccd):
     ccd2idx[j] = i
for i, j in enumerate(unq_bzccd):
     bzcidx[j] = i
for i, j in enumerate(unq_grup_cd):
     grupidx[j] = i    
train_list = [[int(li[0])-1,ccd2idx[str(li[1])], bzcidx[str(li[2])],grupidx[str(li[3])]] for li in rating_list]
# x_train, x_test = train_test_split(train_list, test_size=0.3, shuffle=True, random_state=34)
train_list = pd.DataFrame(train_list, columns = ['sex', 'ccd','bzc','item'])
x_train, x_test = train_test_split(train_list, test_size=0.3, shuffle=True, stratify =train_list[['sex', 'ccd','bzc']], random_state=34)

num_ccd:314 num_bzc:925 num_item:16695


In [15]:
class DfData(torch.utils.data.Dataset):
    def __init__(self,features,num_sex,num_ccd,num_bzc,num_grup,num_ng=0,train_mat=None,is_training=None):
        super(DfData,self).__init__()
        # self.features_ps = torch.Tensor(features).int()
        self.features_ps = features
        self.num_sex = num_sex
        self.num_bzc = num_bzc
        self.num_ccd = num_ccd
        self.num_grup = num_grup
        self.num_ng = num_ng
        self.labels = [1] * len(features)
        self.train_mat = train_mat
        self.is_training = is_training
        
        # negative sampling 
    def set_ng_sample(self):
        assert self.is_training, "no need to sampling when testing"
         
        self.features_ng = []
        for x in self.features_ps:
            # sex
            for _ in range(self.num_ng):
                s = np.random.randint(self.num_sex)
                u = np.random.randint(self.num_ccd)
                q = np.random.randint(self.num_bzc)
                j = np.random.randint(self.num_grup)
                # train set에 있는 경우 다시 뽑기
                self.features_ng.append([s,u,q,j])

        labels_ps = [1] * len(self.features_ps)
        labels_ng = [0] * len(self.features_ng)
        
        self.features_fill = torch.Tensor(self.features_ps + self.features_ng).to(torch.int64)
        self.labels_fill= torch.Tensor(labels_ps + labels_ng)
        

    def __len__(self):
        return (self.num_ng+1) * len(self.labels)
        
    def __getitem__(self,idx):
        features = self.features_fill if self.is_training else torch.Tensor(self.features_ps).to(torch.int64)
        labels = self.labels_fill if self.is_training else torch.Tensor(self.labels)
        user = features[idx]
        label = labels[idx]
        return user, label

In [16]:
num_sex = 2
x_train = x_train.values.tolist()
x_test = x_test.values.tolist()
train_dataset = DfData(x_train, num_sex,num_ccd,num_bzc, num_grup, num_ng=1, is_training=True)
test_dataset = DfData(x_test, num_sex,num_ccd,num_bzc, num_grup)
test_loader = data.DataLoader(test_dataset, batch_size = 128, shuffle=False, num_workers=0)
train_loader = data.DataLoader(train_dataset, batch_size = 128, shuffle=True, num_workers=0)

In [17]:
test_dataset = DfData(x_test, num_sex,num_ccd,num_bzc, num_grup, num_ng=1, is_training=True)
test_loader = data.DataLoader(test_dataset, batch_size = 128, shuffle=True, num_workers=0)

In [19]:
class FeaturesEmbedding(torch.nn.Module):

    def __init__(self, field_dims, embed_dim):
        super().__init__()
        self.embedding = torch.nn.Embedding(sum(field_dims), embed_dim)
        self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.int64())
        torch.nn.init.xavier_uniform_(self.embedding.weight.data)

    def forward(self, x):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        x = x + x.new_tensor(self.offsets).unsqueeze(0)
        return self.embedding(x)

In [20]:
class EulerInteractionLayer(nn.Module):
    def __init__(self,  inshape, outshape, embed_dim):
        super().__init__()
        self.inshape, self.outshape = inshape, outshape
        self.feature_dim = embed_dim


        # Initial assignment of the order vectors, which significantly affects the training effectiveness of the model.
        # We empirically provide two effective initialization methods here.
        # How to better initialize is still a topic to be further explored.
        # Note: 👆 👆 👆
        if inshape == outshape:
            init_orders = torch.eye(inshape // self.feature_dim, outshape // self.feature_dim)
        else:
            init_orders = torch.softmax(torch.randn(inshape // self.feature_dim, outshape // self.feature_dim) / 0.01, dim = 0)
        
        self.inter_orders = nn.Parameter(init_orders)
        self.im = nn.Linear(inshape, outshape)
        nn.init.normal_(self.im.weight , mean = 0 , std = 0.01)

        self.bias_lam = nn.Parameter(torch.randn(1, self.feature_dim, outshape // self.feature_dim) * 0.01)
        self.bias_theta = nn.Parameter(torch.randn(1, self.feature_dim, outshape // self.feature_dim) * 0.01)

        self.drop_ex = nn.Dropout(0.3)
        self.drop_im = nn.Dropout(p = 0.3)
        self.norm_r = nn.LayerNorm([self.feature_dim])
        self.norm_p = nn.LayerNorm([self.feature_dim])
        

    def forward(self, complex_features):
        r, p = complex_features

        lam = r ** 2 + p ** 2 + 1e-8
        theta = torch.atan2(p, r)
        lam, theta = lam.reshape(lam.shape[0], -1, self.feature_dim), theta.reshape(theta.shape[0], -1, self.feature_dim)
        lam = 0.5 * torch.log(lam)
        lam, theta = torch.transpose(lam, -2, -1), torch.transpose(theta, -2, -1)
        lam, theta = self.drop_ex(lam), self.drop_ex(theta)
        lam, theta =  lam @ (self.inter_orders) + self.bias_lam,  theta @ (self.inter_orders) + self.bias_theta
        lam = torch.exp(lam)
        lam, theta = torch.transpose(lam, -2, -1), torch.transpose(theta, -2, -1)

        r, p = r.reshape(r.shape[0], -1), p.reshape(p.shape[0], -1)
        r, p = self.drop_im(r), self.drop_im(p)
        r, p = self.im(r), self.im(p)
        r, p = torch.relu(r), torch.relu(p)
        r, p = r.reshape(r.shape[0], -1, self.feature_dim), p.reshape(p.shape[0], -1, self.feature_dim)
        
        o_r, o_p =  r + lam * torch.cos(theta), p + lam * torch.sin(theta)
        o_r, o_p = o_r.reshape(o_r.shape[0], -1, self.feature_dim), o_p.reshape(o_p.shape[0], -1, self.feature_dim)
        # if self.apply_norm:
        #     o_r, o_p = self.norm_r(o_r), self.norm_p(o_p)
        return o_r, o_p

In [21]:
class EulerNet(torch.nn.Module):
    def __init__(self, field_dims, embed_dim):
        super().__init__()
        self.embedding = FeaturesEmbedding(field_dims, embed_dim)
        field_num = len(field_dims) 
        order_list = [field_num]
        shape_list = [embed_dim * field_num] + [num_neurons * embed_dim for num_neurons in order_list]

        interaction_shapes = []
        for inshape, outshape in zip(shape_list[:-1], shape_list[1:]):
            interaction_shapes.append(EulerInteractionLayer(inshape, outshape,embed_dim))

        self.Euler_interaction_layers = nn.Sequential(*interaction_shapes)
        self.mu = nn.Parameter(torch.ones(1, field_num, 1))
        self.reg = nn.Linear(shape_list[-1], 1)
        nn.init.xavier_normal_(self.reg.weight)

    def forward(self, feature):
        feature = self.embedding(feature)
        r, p = self.mu * torch.cos(feature), self.mu * torch.sin(feature)
        o_r, o_p = self.Euler_interaction_layers((r, p))
        o_r, o_p = o_r.reshape(o_r.shape[0], -1), o_p.reshape(o_p.shape[0], -1)
        re, im = self.reg(o_r), self.reg(o_p)
        self.logits = re + im
        self.output = self.logits.squeeze(1)
        return self.output


In [22]:
sex = 2 
field_dims = [sex, num_ccd,num_bzc,num_grup]
embed_dim= 50
device = 'cuda:0'

In [23]:
net =  EulerNet(field_dims , embed_dim)

In [19]:
net.to(device)

EulerNet(
  (embedding): FeaturesEmbedding(
    (embedding): Embedding(17936, 50)
  )
  (Euler_interaction_layers): Sequential(
    (0): EulerInteractionLayer(
      (im): Linear(in_features=200, out_features=200, bias=True)
      (drop_ex): Dropout(p=0.3, inplace=False)
      (drop_im): Dropout(p=0.3, inplace=False)
      (norm_r): LayerNorm((50,), eps=1e-05, elementwise_affine=True)
      (norm_p): LayerNorm((50,), eps=1e-05, elementwise_affine=True)
    )
  )
  (reg): Linear(in_features=200, out_features=1, bias=True)
)

In [24]:
learning_rate = 0.005
epochs = 50
loss_function = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [22]:
for ep in range(epochs):   
    avg_cost = 0
    train_loader.dataset.set_ng_sample()
    for user, label in train_loader:
        user = user.to(torch.int64).to(device)
        label = label.to('cuda:0')
        net.zero_grad()
        pred = net(user)
        loss = loss_function(pred, label)
        loss.backward()
        optimizer.step()
        avg_cost += loss / len(train_loader)
    net.eval()
    # HR = metrics(net, test_loader, 10)    
    # print(np.mean(HR))
    print('Epoch:', '%04d' % (ep + 1), 'cost =', '{:.9f}'.format(avg_cost)) 

Epoch: 0001 cost = 0.183817670
Epoch: 0002 cost = 0.079450287
Epoch: 0003 cost = 0.054364301
Epoch: 0004 cost = 0.042966928
Epoch: 0005 cost = 0.035959858
Epoch: 0006 cost = 0.031221243
Epoch: 0007 cost = 0.027339574
Epoch: 0008 cost = 0.024981735
Epoch: 0009 cost = 0.022988668
Epoch: 0010 cost = 0.020968203
Epoch: 0011 cost = 0.019300494
Epoch: 0012 cost = 0.018501198
Epoch: 0013 cost = 0.017513894
Epoch: 0014 cost = 0.016814334


KeyboardInterrupt: 

In [25]:
# model_path = "C:/Users/db/Desktop/eurlent/eurlent.pth"

# # 모델을 로드합니다.
# net = torch.load(model_path)

In [26]:
x_train = np.array(x_train)
x_test = np.array(x_test)

In [27]:
train_dict = {}
for item in x_train:
    key = tuple(item[:3])  # 2, 91, 750을 튜플로 만듭니다.
    value = item[3]  # 나머지 값은 딕셔너리의 값으로 사용합니다.
    
    if key in train_dict:
        train_dict[key].append(value)
    else:
        train_dict[key] = [value]

In [28]:
result_dict = {}
for item in x_test:
    key = tuple(item[:3])  # 2, 91, 750을 튜플로 만듭니다.
    value = item[3]  # 나머지 값은 딕셔너리의 값으로 사용합니다.
    
    if key in result_dict:
        result_dict[key].append(value)
    else:
        result_dict[key] = [value]
keys_list = list(train_dict.keys())

In [29]:
# test 셋 비율확인 
dic2 = []
for i in result_dict.keys():
    dic2.append(len(result_dict[i]))
# 평균적인 과목의 개수 
(sum(dic2))/len(dic2)

27.749895833333333

In [30]:
dic = []
for i in train_dict.keys():
    dic.append(len(train_dict[i]))
# 평균적인 과목의 개수 
(sum(dic))/len(dic)

64.7496875

In [29]:
from tqdm import tqdm
import torch

def calculate_ap(gt, pred, k):
    """
    단일 클래스에 대한 평균 정밀도 (AP)를 계산합니다.
    """
    precision_at_k = 0.0
    true_positives = 0

    for i in range(k):
        if pred[i] in gt:
            true_positives += 1
            precision_at_k += true_positives / (i + 1)

    if not gt:
        return 0.0

    return precision_at_k / len(gt)

def mean_average_precision(k):
    result = []

    for i in tqdm(keys_list, desc="mAP 계산 중", position=0, leave=True):
        try:
            b = result_dict[i]
            c = train_dict[i]
        except KeyError:
            continue

        p = []
        for j in range(num_grup):
            p.append(list(i) + [j])
        p = torch.tensor(p).to(device)
        pred = net(p).to(torch.int)
        values, sorted_indices = torch.sort(pred, descending=True)

        exclusive_indices = [idx for idx in sorted_indices.tolist() if idx not in set(c)] 
        exclusive_indices = exclusive_indices[:k]

        ap = calculate_ap(b, exclusive_indices, k)
        result.append(ap)

    mean_ap = sum(result) / len(result) if result else 0.0
    return mean_ap


In [30]:
map_10 = mean_average_precision(10)
map_25 = mean_average_precision(25)
map_50 = mean_average_precision(50)
map_100 = mean_average_precision(100)

mAP 계산 중: 100%|█████████████████████████████████████████████████████████████████| 9600/9600 [11:17<00:00, 14.17it/s]


In [31]:
print(map_10)
print(map_25)
print(map_50)
print(map_100)

0.05588498331850875
0.08214926542347363
0.10114489021418158
0.11784293368635597


In [ ]:
from tqdm import tqdm  # Import tqdm for the progress bar

def precision(k):
    result = []
    for i in tqdm(keys_list, desc="Calculating Precision", position=0, leave=True):  # tqdm setup
        p = []
        try:
            b = result_dict[i]
            c = train_dict[i]
        except KeyError:
            continue
        for j in range(num_grup):
            p.append(list(i) + [j])  # Fix variable name from 'a' to 'i'
        p = torch.tensor(p).to(device)
        pred = net(p).to(torch.int)  # Change to 'torch.int' for data type
        values, sorted_indices = torch.sort(pred, descending=True)
        
        exclusive_indices = [idx for idx in sorted_indices.tolist() if idx not in set(c)] 
        exclusive_indices = exclusive_indices[:k]
        intersection_count = len(set(b).intersection(set(exclusive_indices)))
        result.append(intersection_count / k)
    
    return result

In [67]:
for i in tqdm(keys_list[:1], desc="Calculating Precision", position=0, leave=True):
    p = []
    try:
        b = result_dict[i]
        c = train_dict[i]
    except KeyError:
        continue
    
    for j in range(num_grup):
        p.append(list(i) + [j])  # Fix variable name from 'a' to 'i'
    
    p = torch.tensor(p).to(device)
    pred = net(p).to(torch.int)  # Change to 'torch.int' for data type
    values, sorted_indices = torch.sort(pred, descending=True)
    print(sorted_indices[:30])
    original_indices = [idx for idx in sorted_indices.tolist() if idx not in set(c)]
    print(original_indices[:10])

Calculating Precision: 100%|█████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.71it/s]

tensor([1936, 1944, 1952, 1992,   69,  282, 1846, 1940, 1947, 1951, 1956, 1957,
        1960, 1962, 1963, 1964, 1965, 1969, 1970, 1972, 1975, 1976, 1981, 1984,
        1985, 1986, 1987, 1988, 1993, 1997], device='cuda:0')
[1936, 69, 1846, 1969, 1972, 1986, 1988, 1993, 1997, 51]


In [55]:
from tqdm import tqdm  # Import tqdm for the progress bar

def precision_and_map(k_values):
    precision_results = {k: [] for k in k_values}
    map_results = {k: [] for k in k_values}
    recall_results = {k: [] for k in k_values}
    
    for i in tqdm(keys_list, desc="Calculating Precision", position=0, leave=True):
        p = []
        try:
            b = result_dict[i]
            c = train_dict[i]
        except KeyError:
            continue
        
        for j in range(num_grup):
            p.append(list(i) + [j])  # Fix variable name from 'a' to 'i'
        
        p = torch.tensor(p).to(device)
        pred = net(p).to(torch.int)  # Change to 'torch.int' for data type
        values, sorted_indices = torch.sort(pred, descending=True)
        
        original_indices = [idx for idx in sorted_indices.tolist() if idx not in set(c)]
        
        for k in k_values:
            exclusive_indices = original_indices[:k]  # Use the original indices here
            # Precision 계산
            intersection_count_k = len(set(b).intersection(set(exclusive_indices)))
            precision_results[k].append(intersection_count_k / k)
            # recall 계산 
            recall_results[k].append(intersection_count_k / len(b))
            # MAP 계산
            precision_at_i = []
            
            for rank, idx in enumerate(exclusive_indices, start=1):
                if idx in set(b):
                    precision_at_i.append(sum([1 if r in set(b) else 0 for r in exclusive_indices[:rank]]) / rank)
            
            if precision_at_i:
                map_results[k].append(sum(precision_at_i) / len(precision_at_i))
            else:
                map_results[k].append(0.0)
    return precision_results, map_results, recall_results

k_values = [10, 25, 50, 100]
precision_results, map_results, recall_results = precision_and_map(k_values)

# Access the results
precision_result_10 = precision_results[10]
precision_result_25 = precision_results[25]
precision_result_50 = precision_results[50]
precision_result_100 = precision_results[100]

map_result_10 = map_results[10]
map_result_25 = map_results[25]
map_result_50 = map_results[50]
map_result_100 = map_results[100]

recall_result_10 = recall_results[10]
recall_result_25 = recall_results[25]
recall_result_50 = recall_results[50]
recall_result_100 = recall_results[100]


Calculating Precision: 100%|███████████████████████████████████████████████████████| 9600/9600 [12:54<00:00, 12.39it/s]


In [56]:
precision_10_mean = np.mean(precision_result_10)
precision_25_mean = np.mean(precision_result_25)
precision_50_mean = np.mean(precision_result_50)
precision_100_mean = np.mean(precision_result_100)

recall_10_mean = np.mean(recall_result_10)
recall_25_mean = np.mean(recall_result_25)
recall_50_mean = np.mean(recall_result_50)
recall_100_mean = np.mean(recall_result_100)

map_10_mean = np.mean(map_result_10)
map_25_mean = np.mean(map_result_25)
map_50_mean = np.mean(map_result_50)
map_100_mean = np.mean(map_result_100)

# 평균 출력
print("Precision@10 Mean:", precision_10_mean)
print("Precision@25 Mean:", precision_25_mean)
print("Precision@50 Mean:", precision_50_mean)
print("Precision@100 Mean:", precision_100_mean)

print("Recall@10 Mean:", recall_10_mean)
print("Recall@25 Mean:", recall_25_mean)
print("Recall@50 Mean:", recall_50_mean)
print("Recall@100 Mean:", recall_100_mean)

print("MAP@10 Mean:", map_10_mean)
print("MAP@25 Mean:", map_25_mean)
print("MAP@50 Mean:", map_50_mean)
print("MAP@100 Mean:", map_100_mean)

Precision@10 Mean: 0.198125
Precision@25 Mean: 0.15697083333333336
Precision@50 Mean: 0.1217916666666667
Precision@100 Mean: 0.08877291666666667
Recall@10 Mean: 0.10721837772174389
Recall@25 Mean: 0.21233277465106365
Recall@50 Mean: 0.3271400957263754
Recall@100 Mean: 0.4728926174633057
MAP@10 Mean: 0.3632200193426136
MAP@25 Mean: 0.3096626697258589
MAP@50 Mean: 0.2635194162559319
MAP@100 Mean: 0.2232019756896978


In [ ]:
# Example usage:
a_10 = precision(10)
a_25 = precision(25)
a_50 = precision(50)
a_100 = precision(100)

In [59]:
print(np.array(a_10).mean())
print(np.array(a_25).mean())
print(np.array(a_50).mean())
print(np.array(a_100).mean())

0.18898958333333335
0.14987083333333337
0.11610416666666667
0.08527187500000001


In [60]:
from tqdm import tqdm  # Import tqdm for the progress bar

def Recall(k):
    result = []
    for i in tqdm(keys_list, desc="Calculating Precision", position=0, leave=True):  # tqdm setup
        p = []
        try:
            b = result_dict[i]
            c = train_dict[i]
        except KeyError:
            continue
        for j in range(num_grup):
            p.append(list(i) + [j])  # Fix variable name from 'a' to 'i'
        
        p = torch.tensor(p).to(device)
        pred = net(p).to(torch.int)  # Change to 'torch.int' for data type
        values, sorted_indices = torch.sort(pred, descending=True)
        
        exclusive_indices = [idx for idx in sorted_indices.tolist() if idx not in set(c)]
        exclusive_indices = exclusive_indices[:k]
        
        intersection_count = len(set(b).intersection(set(exclusive_indices)))
        result.append(intersection_count / len(set(b)))
    
    return result

In [61]:
b_10 = Recall(10)
b_25 = Recall(25)
b_50 = Recall(50)
b_100 = Recall(100)

Calculating Precision: 100%|███████████████████████████████████████████████████████| 9600/9600 [15:33<00:00, 10.29it/s]


In [62]:
print(np.array(b_10).mean())
print(np.array(b_25).mean())
print(np.array(b_50).mean())
print(np.array(b_100).mean())

0.10634116829347541
0.20982420153029313
0.3227293928245148
0.4700457197510236
